In [1]:
import torch
import onnx
from onnx import numpy_helper
from onnx2torch import convert
import tqdm
import numpy as np
import gc

In [2]:
# !pip install onnx2torch

In [3]:
### load ds
from face_lib.dataset_classes.lightning_datasets import MXFaceDataset

vgg_ds = MXFaceDataset("/app/datasets/faces_vgg_112x112")
# softmax_weights_path = '/app/model_weights/softmax_weight.pt'
# softmax_weights = torch.load(softmax_weights_path)
# softmax_weights_norm = torch.norm(
#             softmax_weights, dim=1, keepdim=True
#         )  # [N, 512]
# softmax_weights = (
#             softmax_weights / softmax_weights_norm
#         )

/home/kolesnikov/miniconda/lib/python3.9/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [4]:
onnx_model = onnx.load("/app/model_weights/backbone/vgg_iresnet50/vgg2_r50.onnx")

In [5]:
torch_model = convert(onnx_model)

In [6]:
labels = np.load("/app/datasets/faces_vgg_112x112/labels.npy")

In [7]:
labels_tuple = np.unique(labels, return_counts = True)

In [11]:
print(labels_tuple[1][5])

259


In [12]:
torch_model.to("cuda:0")

GraphModule(
  (Conv_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (initializers): Module()
  (PRelu_1): OnnxPReLU()
  (BatchNormalization_2): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
  (Conv_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (PRelu_4): OnnxPReLU()
  (Conv_5): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Conv_6): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
  (Add_7): OnnxBinaryMathOperation()
  (BatchNormalization_8): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
  (Conv_9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (PRelu_10): OnnxPReLU()
  (Conv_11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Add_12): OnnxBinaryMathOperation()
  (BatchNormalization_13): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.

Усреднение эмбедингов:

In [13]:
torch_model.eval()

GraphModule(
  (Conv_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (initializers): Module()
  (PRelu_1): OnnxPReLU()
  (BatchNormalization_2): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
  (Conv_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (PRelu_4): OnnxPReLU()
  (Conv_5): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (Conv_6): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
  (Add_7): OnnxBinaryMathOperation()
  (BatchNormalization_8): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.10000002384185791, affine=True, track_running_stats=True)
  (Conv_9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (PRelu_10): OnnxPReLU()
  (Conv_11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (Add_12): OnnxBinaryMathOperation()
  (BatchNormalization_13): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.

In [14]:
sfw_vgg = []
current_ind = 0

In [15]:
for i in tqdm.tqdm(range(len(labels_tuple[0]))):
    with torch.no_grad():
        batch_for_label = torch.cat([vgg_ds[current_ind + j][0][None, ...] for j in range(labels_tuple[1][i])]).to("cuda:0")
        emb_batch = torch_model(batch_for_label)
        #print(emb_batch.shape)
        mean_emb = torch.mean(emb_batch, dim = 0)
    sfw_vgg.append(mean_emb.to("cpu"))
    current_ind += labels_tuple[1][i]
    torch.cuda.empty_cache()
    gc.collect()

100%|██████████| 8631/8631 [2:24:41<00:00,  1.01s/it]  


In [16]:
sfw_vgg_torch = torch.stack(sfw_vgg)

In [17]:
sfw_vgg_torch.shape

torch.Size([8631, 512])

In [19]:
torch.save(sfw_vgg_torch, "/app/model_weights/backbone/vgg_iresnet50/softmax_weights_mean_vgg.pt")

In [20]:
sfw_vgg_normalised = torch.nn.functional.normalize(sfw_vgg_torch, p=2.0, dim=1)

In [21]:
torch.save(sfw_vgg_normalised, "/app/model_weights/backbone/vgg_iresnet50/softmax_weights_mean_vgg_normalised.pt")

In [49]:
### load model
from face_lib.models.lightning_wrappers import ResNet

backbone_old = ResNet(
    "iresnet50_normalized",
    "/app/model_weights/backbone/ms1mv3_arcface_r50/backbone.pth",
    learnable=False,
)

/app/face_lib/models/lightning_wrappers.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  backbone_dict = torch.load(weights)


In [12]:
### predict emb
# x = vgg_ds[0][0][None, ...]
id = 0
x = torch.cat([vgg_ds[2000][0][None, ...], vgg_ds[1][0][None, ...]])
torch_model.to("cpu")
backbone_outputs = torch_model(x)
backbone_outputs = torch.nn.functional.normalize(backbone_outputs, p=2.0, dim = 1)
emb = backbone_outputs

In [24]:
vgg_ds[2000][1]

tensor(6)

In [13]:
sfw_vgg_normalised = torch.load("/app/model_weights/backbone/vgg_iresnet50/softmax_weights_mean_vgg_normalised.pt")

/tmp/ipykernel_852202/1228789056.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sfw_vgg_normalised = torch.load("/app/model_weights/backbone/vgg_iresnet50/softmax_weigh

In [14]:
import numpy as np

cos = []
for weight in sfw_vgg_normalised:
    cos.append(weight.detach().numpy() @ emb[0].detach().numpy())
cos = np.array(cos)

In [25]:
print(np.max(cos))

0.49048865


In [64]:
max(cos)

18.790724

In [58]:
np.max(cos)
np.min(cos)

-0.1538214

In [12]:
len(vgg_ds)

3137807

Saving Softmax Weights from model to separate file

In [50]:
model_trained = torch.load("/app/outputs/scf_new_data/arcface_vgg2/epoch=19-step=122560.ckpt")

/tmp/ipykernel_1139223/632247186.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_trained = torch.load("/app/outputs/scf_new_data/arcface_vgg2/epoch=19-step=122560.

In [51]:
model_trained.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])

In [52]:
sw_trained = model_trained["state_dict"]["softmax_weights"]

In [53]:
torch.save(sw_trained, "/app/model_weights/backbone/vgg_iresnet50/softmax_weights_trained_correct.pt")

Testing new softmax weights

In [65]:
### predict emb
# x = vgg_ds[0][0][None, ...]
id = 0
x = torch.cat([vgg_ds[2000347][0][None, ...], vgg_ds[1][0][None, ...]])
torch_model.to("cpu")
backbone_new_outputs = torch_model(x)
backbone_new_outputs = torch.nn.functional.normalize(backbone_new_outputs, p=2.0, dim = 1)
backbone_old_outputs = backbone_old(x)

In [66]:
sw_trained = torch.nn.functional.normalize(sw_trained, p=2.0, dim = 1)

In [67]:
import numpy as np

cos_new = []
cos_old = []
for weight in sw_trained:
    cos_new.append(weight.detach().cpu().numpy() @ backbone_new_outputs[0].detach().cpu().numpy())
    cos_old.append(weight.detach().cpu().numpy() @ backbone_old_outputs["feature"][0].detach().cpu().numpy())
cos_new = np.array(cos_new)
cos_old = np.array(cos_old)

In [68]:
np.argmax(cos_new)

5460

In [69]:
np.max(cos_new)

0.38555342

In [59]:
print(np.argmax(cos_old))
print(np.max(cos_old))

2024
0.15234926


In [43]:
vgg_ds[200345][1]

tensor(543)

In [ ]:
for image in vgg_ds:
    pass

In [7]:
from torch.utils.data import DataLoader

In [8]:
dl = DataLoader(
            vgg_ds,
            batch_size=512,
            drop_last=False,
            shuffle=False,
            num_workers=40,
        )

In [9]:
for batch in tqdm.tqdm(dl):
    pass

100%|██████████| 6129/6129 [01:33<00:00, 65.75it/s]


Смотрим распределение получившихся косинусных расстояний

In [ ]:
vgg_ds = MXFaceDataset("/app/datasets/faces_vgg_112x112")
labels = np.load("/app/datasets/ms1m/labels.npy")